In [6]:
# Variables - file paths
tif_path = "../workspace/FWI data/2024_Brazil_LandCover.tif"
nc_path = "../workspace/hazards/Fire/FWI/ensemble_return_period_FWI_final.nc"


In [7]:
import rasterio
import numpy as np
import netCDF4 as nc
from scipy import stats
import os
from pathlib import Path
import time


In [8]:
# Read TIF file metadata (without loading data into memory)
print(f"Reading TIF file metadata: {tif_path}")
print(f"File exists: {os.path.exists(tif_path)}")

with rasterio.open(tif_path) as src:
    print(f"\nTIF file properties:")
    print(f"  Width: {src.width}")
    print(f"  Height: {src.height}")
    print(f"  Number of bands: {src.count}")
    print(f"  CRS: {src.crs}")
    print(f"  Bounds: {src.bounds}")
    print(f"  Transform: {src.transform}")
    print(f"  Data type: {src.dtypes[0]}")
    
    # Calculate resolution from transform
    transform = src.transform
    pixel_size_x = abs(transform[0])  # Resolution in x direction
    pixel_size_y = abs(transform[4])  # Resolution in y direction
    print(f"\nSpatial resolution:")
    print(f"  Pixel size X: {pixel_size_x} degrees")
    print(f"  Pixel size Y: {pixel_size_y} degrees")


Reading TIF file metadata: ../workspace/FWI data/2024_Brazil_LandCover.tif
File exists: True

TIF file properties:
  Width: 167551
  Height: 144835
  Number of bands: 1
  CRS: EPSG:4326
  Bounds: BoundingBox(left=-73.99593675196482, bottom=-33.75581376830244, right=-28.84184950111183, top=5.2764344843328335)
  Transform: | 0.00, 0.00,-74.00|
| 0.00,-0.00, 5.28|
| 0.00, 0.00, 1.00|
  Data type: uint8

Spatial resolution:
  Pixel size X: 0.00026949458523585647 degrees
  Pixel size Y: 0.00026949458523585647 degrees


In [9]:
# Load and inspect the NetCDF file structure
print(f"Loading NetCDF file: {nc_path}")
print(f"File exists: {os.path.exists(nc_path)}")

with nc.Dataset(nc_path, 'r') as ds:
    print("\n" + "="*60)
    print("NETCDF FILE STRUCTURE")
    print("="*60)
    
    print(f"\nFile format: {ds.file_format}")
    print(f"File mode: {ds.data_model}")
    
    print(f"\n{'='*60}")
    print("DIMENSIONS:")
    print("="*60)
    for dim_name, dim_obj in ds.dimensions.items():
        print(f"  {dim_name}: {dim_obj.size} (unlimited: {dim_obj.isunlimited()})")
    
    print(f"\n{'='*60}")
    print("VARIABLES:")
    print("="*60)
    for var_name, var_obj in ds.variables.items():
        print(f"\n  Variable: {var_name}")
        print(f"    Shape: {var_obj.shape}")
        print(f"    Dimensions: {var_obj.dimensions}")
        print(f"    Data type: {var_obj.dtype}")
        print(f"    Attributes: {list(var_obj.ncattrs())}")
        if var_obj.ncattrs():
            for attr in var_obj.ncattrs():
                print(f"      {attr}: {getattr(var_obj, attr)}")
    
    print(f"\n{'='*60}")
    print("GLOBAL ATTRIBUTES:")
    print("="*60)
    for attr_name in ds.ncattrs():
        print(f"  {attr_name}: {getattr(ds, attr_name)}")
    
    print(f"\n{'='*60}")
    print("COORDINATE VARIABLES (sample values):")
    print("="*60)
    # Try to identify coordinate variables
    dim_names = list(ds.dimensions.keys())
    for dim_name in dim_names:
        if dim_name in ds.variables:
            var_obj = ds.variables[dim_name]
            if len(var_obj.shape) == 1 and var_obj.size <= 20:
                values = var_obj[:]
                print(f"  {dim_name}: {values}")
            elif len(var_obj.shape) == 1:
                values = var_obj[:]
                print(f"  {dim_name}: [{values[0]}, ..., {values[-1]}] (size: {len(values)})")
    
    print(f"\n{'='*60}")
    print("DATA VARIABLE SUMMARY:")
    print("="*60)
    # Find main data variable (exclude coordinate variables)
    data_vars = [v for v in ds.variables.keys() if v not in dim_names]
    for var_name in data_vars:
        var_obj = ds.variables[var_name]
        if len(var_obj.shape) >= 2:  # Likely a data variable
            print(f"\n  {var_name}:")
            try:
                # Try to get a sample slice
                sample_idx = tuple(0 for _ in var_obj.shape)
                sample_value = var_obj[sample_idx]
                print(f"    Sample value at {sample_idx}: {sample_value}")
                print(f"    Shape: {var_obj.shape}")
            except Exception as e:
                print(f"    Could not read sample: {e}")


Loading NetCDF file: ../workspace/hazards/Fire/FWI/ensemble_return_period_FWI_final.nc
File exists: True

NETCDF FILE STRUCTURE

File format: NETCDF4
File mode: NETCDF4

DIMENSIONS:
  GWL: 4 (unlimited: False)
  lon: 168 (unlimited: False)
  lat: 162 (unlimited: False)
  return_period: 5 (unlimited: False)
  ensemble: 4 (unlimited: False)

VARIABLES:

  Variable: GWL
    Shape: (4,)
    Dimensions: ('GWL',)
    Data type: <class 'str'>
    Attributes: ['long_name']
      long_name: Global Warming Level

  Variable: lon
    Shape: (168,)
    Dimensions: ('lon',)
    Data type: float64
    Attributes: ['_FillValue']
      _FillValue: nan

  Variable: lat
    Shape: (162,)
    Dimensions: ('lat',)
    Data type: float64
    Attributes: ['_FillValue']
      _FillValue: nan

  Variable: return_period
    Shape: (5,)
    Dimensions: ('return_period',)
    Data type: int64
    Attributes: []

  Variable: ensemble
    Shape: (4,)
    Dimensions: ('ensemble',)
    Data type: <class 'str'>
    A

In [ ]:
# Compare resolutions between TIF and NetCDF files
print("="*60)
print("RESOLUTION COMPARISON")
print("="*60)

# Get TIF resolution
with rasterio.open(tif_path) as tif_src:
    tif_transform = tif_src.transform
    tif_res_x = abs(tif_transform[0])
    tif_res_y = abs(tif_transform[4])
    tif_bounds = tif_src.bounds
    tif_width = tif_src.width
    tif_height = tif_src.height
    tif_crs = tif_src.crs

print(f"\nTIF File:")
print(f"  Dimensions: {tif_width} x {tif_height}")
print(f"  Resolution X: {tif_res_x:.8f} degrees")
print(f"  Resolution Y: {tif_res_y:.8f} degrees")
print(f"  Bounds: {tif_bounds}")
print(f"  CRS: {tif_crs}")

# Get NetCDF resolution
with nc.Dataset(nc_path, 'r') as nc_ds:
    # Find lon/lat dimensions
    dim_names = list(nc_ds.dimensions.keys())
    lon_dim = next((d for d in dim_names if d.lower() in ['lon', 'longitude', 'x']), None)
    lat_dim = next((d for d in dim_names if d.lower() in ['lat', 'latitude', 'y']), None)
    
    if lon_dim and lat_dim:
        # Get coordinate values
        if lon_dim in nc_ds.variables:
            lon_vals = nc_ds.variables[lon_dim][:]
        else:
            lon_vals = np.arange(nc_ds.dimensions[lon_dim].size)
            
        if lat_dim in nc_ds.variables:
            lat_vals = nc_ds.variables[lat_dim][:]
        else:
            lat_vals = np.arange(nc_ds.dimensions[lat_dim].size)
        
        # Calculate resolution
        if len(lon_vals) > 1:
            nc_res_x = abs(lon_vals[1] - lon_vals[0])
        else:
            nc_res_x = None
            
        if len(lat_vals) > 1:
            nc_res_y = abs(lat_vals[1] - lat_vals[0])
        else:
            nc_res_y = None
        
        nc_width = len(lon_vals)
        nc_height = len(lat_vals)
        nc_bounds = (float(lon_vals.min()), float(lat_vals.min()), 
                     float(lon_vals.max()), float(lat_vals.max()))
        
        print(f"\nNetCDF File:")
        print(f"  Dimensions: {nc_width} x {nc_height}")
        if nc_res_x is not None:
            print(f"  Resolution X: {nc_res_x:.8f} degrees")
        if nc_res_y is not None:
            print(f"  Resolution Y: {nc_res_y:.8f} degrees")
        print(f"  Bounds: {nc_bounds}")
        print(f"  Coordinate variables: lon={lon_dim}, lat={lat_dim}")
        
        # Compare
        print(f"\n{'='*60}")
        print("COMPARISON:")
        print("="*60)
        
        if nc_res_x is not None and nc_res_y is not None:
            res_diff_x = abs(tif_res_x - nc_res_x)
            res_diff_y = abs(tif_res_y - nc_res_y)
            
            print(f"\nResolution differences:")
            print(f"  X: {res_diff_x:.10f} degrees")
            print(f"  Y: {res_diff_y:.10f} degrees")
            
            if res_diff_x < 1e-6 and res_diff_y < 1e-6:
                print(f"\n✓ Resolutions match!")
            else:
                print(f"\n⚠ Resolutions differ")
                print(f"  TIF resolution is {tif_res_x/nc_res_x:.4f}x in X direction")
                print(f"  TIF resolution is {tif_res_y/nc_res_y:.4f}x in Y direction")
        
        # Compare dimensions
        print(f"\nDimension comparison:")
        print(f"  TIF: {tif_width} x {tif_height} = {tif_width * tif_height:,} pixels")
        print(f"  NC:  {nc_width} x {nc_height} = {nc_width * nc_height:,} pixels")
        print(f"  Ratio: {tif_width/nc_width:.4f} x {tif_height/nc_height:.4f}")
        
        # Check bounds overlap
        print(f"\nBounds comparison:")
        print(f"  TIF: lon [{tif_bounds.left:.4f}, {tif_bounds.right:.4f}], "
              f"lat [{tif_bounds.bottom:.4f}, {tif_bounds.top:.4f}]")
        print(f"  NC:  lon [{nc_bounds[0]:.4f}, {nc_bounds[2]:.4f}], "
              f"lat [{nc_bounds[1]:.4f}, {nc_bounds[3]:.4f}]")
        
        # Check if bounds overlap
        lon_overlap = not (tif_bounds.right < nc_bounds[0] or nc_bounds[2] < tif_bounds.left)
        lat_overlap = not (tif_bounds.top < nc_bounds[1] or nc_bounds[3] < tif_bounds.bottom)
        
        if lon_overlap and lat_overlap:
            print(f"  ✓ Bounds overlap")
        else:
            print(f"  ⚠ Bounds do not overlap")
    else:
        print(f"\n⚠ Could not find lon/lat dimensions in NetCDF file")
        print(f"  Available dimensions: {dim_names}")


RESOLUTION COMPARISON

TIF File:
  Dimensions: 167551 x 144835
  Resolution X: 0.00026949 degrees
  Resolution Y: 0.00026949 degrees
  Bounds: BoundingBox(left=-73.99593675196482, bottom=-33.75581376830244, right=-28.84184950111183, top=5.2764344843328335)
  CRS: EPSG:4326

NetCDF File:
  Dimensions: 168 x 162
  Resolution X: 0.25000000 degrees
  Resolution Y: 0.25000000 degrees
  Bounds: (-74.125, -34.125, -32.375, 6.125)
  Coordinate variables: lon=lon, lat=lat

COMPARISON:

Resolution differences:
  X: 0.2497305054 degrees
  Y: 0.2497305054 degrees

⚠ Resolutions differ
  TIF resolution is 0.0011x in X direction
  TIF resolution is 0.0011x in Y direction

Dimension comparison:
  TIF: 167551 x 144835 = 24,267,249,085 pixels
  NC:  168 x 162 = 27,216 pixels
  Ratio: 997.3274 x 894.0432

Bounds comparison:
  TIF: lon [-73.9959, -28.8418], lat [-33.7558, 5.2764]
  NC:  lon [-74.1250, -32.3750], lat [-34.1250, 6.1250]
  ✓ Bounds overlap


In [10]:
# Extract days_danger_total and FWI_max from NetCDF and save to separate files
print("="*60)
print("EXTRACTING NETCDF VARIABLES")
print("="*60)

# Output directories - one per variable
output_nc_dir_fwi = Path("../tests/tests_data/hazards/Fire/FWI")
output_nc_dir_fwi.mkdir(parents=True, exist_ok=True)
output_nc_path_fwi = output_nc_dir_fwi / "ensemble_return_period.nc"

output_nc_dir_days = Path("../tests/tests_data/hazards/Fire/days_danger_total")
output_nc_dir_days.mkdir(parents=True, exist_ok=True)
output_nc_path_days = output_nc_dir_days / "ensemble_return_period.nc"

print(f"\nOutput directories:")
print(f"  FWI_max: {output_nc_path_fwi}")
print(f"  days_danger_total: {output_nc_path_days}")

# Open source NetCDF
with nc.Dataset(nc_path, 'r') as src_ds:
    # Check available variables
    print(f"\nAvailable data variables in source:")
    dim_names = list(src_ds.dimensions.keys())
    data_vars = [v for v in src_ds.variables.keys() if v not in dim_names]
    for var_name in data_vars:
        var_obj = src_ds.variables[var_name]
        print(f"  {var_name}: shape {var_obj.shape}, dtype {var_obj.dtype}")
    
    # Verify target variables exist
    if 'days_danger_total' not in data_vars:
        print(f"\n⚠ Warning: 'days_danger_total' not found in source file")
        print(f"  Available variables: {data_vars}")
    if 'FWI_max' not in data_vars:
        print(f"\n⚠ Warning: 'FWI_max' not found in source file")
        print(f"  Available variables: {data_vars}")
    
    # Function to create a NetCDF file with a single data variable
    def create_nc_file_with_variable(output_path, var_name, src_ds):
        """Create a new NetCDF file containing only one data variable."""
        with nc.Dataset(output_path, 'w', format='NETCDF4') as dst_ds:
            # Copy dimensions
            for dim_name, dim_obj in src_ds.dimensions.items():
                if dim_obj.isunlimited():
                    dst_ds.createDimension(dim_name, None)
                else:
                    dst_ds.createDimension(dim_name, dim_obj.size)
            
            # Copy coordinate variables and their attributes
            for coord_var_name in dim_names:
                if coord_var_name in src_ds.variables:
                    src_var = src_ds.variables[coord_var_name]
                    fill_val = getattr(src_var, '_FillValue', None)
                    dst_var = dst_ds.createVariable(
                        coord_var_name, 
                        src_var.dtype, 
                        src_var.dimensions,
                        fill_value=fill_val
                    )
                    dst_var[:] = src_var[:]
                    
                    # Copy attributes (skip _FillValue as it's already set)
                    for attr in src_var.ncattrs():
                        if attr != '_FillValue':
                            setattr(dst_var, attr, getattr(src_var, attr))
            
            # Copy the target data variable
            if var_name in src_ds.variables:
                src_var = src_ds.variables[var_name]
                
                # Create variable in destination
                dst_var = dst_ds.createVariable(
                    var_name, 
                    src_var.dtype, 
                    src_var.dimensions,
                    fill_value=getattr(src_var, '_FillValue', None)
                )
                
                # Copy data
                dst_var[:] = src_var[:]
                
                # Copy attributes (skip _FillValue as it's already set)
                for attr in src_var.ncattrs():
                    if attr != '_FillValue':
                        setattr(dst_var, attr, getattr(src_var, attr))
            
            # Copy global attributes
            for attr in src_ds.ncattrs():
                setattr(dst_ds, attr, getattr(src_ds, attr))
    
    # Create file for FWI_max
    if 'FWI_max' in data_vars:
        print(f"\nCreating NetCDF file for FWI_max...")
        print(f"  Output: {output_nc_path_fwi}")
        create_nc_file_with_variable(output_nc_path_fwi, 'FWI_max', src_ds)
        print(f"  ✓ Created NetCDF file for FWI_max")
        print(f"    Variables: ['GWL', 'lon', 'lat', 'return_period', 'ensemble', 'FWI_max']")
    else:
        print(f"\n⚠ Skipping FWI_max (variable not found)")
    
    # Create file for days_danger_total
    if 'days_danger_total' in data_vars:
        print(f"\nCreating NetCDF file for days_danger_total...")
        print(f"  Output: {output_nc_path_days}")
        create_nc_file_with_variable(output_nc_path_days, 'days_danger_total', src_ds)
        print(f"  ✓ Created NetCDF file for days_danger_total")
        print(f"    Variables: ['GWL', 'lon', 'lat', 'return_period', 'ensemble', 'days_danger_total']")
    else:
        print(f"\n⚠ Skipping days_danger_total (variable not found)")


EXTRACTING NETCDF VARIABLES

Output directories:
  FWI_max: ../tests/tests_data/hazards/Fire/FWI/ensemble_return_period.nc
  days_danger_total: ../tests/tests_data/hazards/Fire/days_danger_total/ensemble_return_period.nc

Available data variables in source:
  FWI_mean: shape (4, 4, 162, 168, 5), dtype float32
  FWI_max: shape (4, 4, 162, 168, 5), dtype float32
  days_low: shape (4, 4, 162, 168, 5), dtype float32
  days_moderate: shape (4, 4, 162, 168, 5), dtype float32
  days_high: shape (4, 4, 162, 168, 5), dtype float32
  days_very_high: shape (4, 4, 162, 168, 5), dtype float32
  days_extreme: shape (4, 4, 162, 168, 5), dtype float32
  days_danger_total: shape (4, 4, 162, 168, 5), dtype float32

Creating NetCDF file for FWI_max...
  Output: ../tests/tests_data/hazards/Fire/FWI/ensemble_return_period.nc
  ✓ Created NetCDF file for FWI_max
    Variables: ['GWL', 'lon', 'lat', 'return_period', 'ensemble', 'FWI_max']

Creating NetCDF file for days_danger_total...
  Output: ../tests/tests

In [ ]:
# Downsample TIF by factor of 4 (using mode for categorical data)
print("="*60)
print("DOWNSAMPLING TIF FILE")
print("="*60)

# Output directory
output_tif_dir = Path("../tests/tests_data/hazards/Fire/land_cover")
output_tif_dir.mkdir(parents=True, exist_ok=True)
output_tif_path = output_tif_dir / "2024_brazil_land_cover.tif"

print(f"\nOutput directory: {output_tif_dir}")
print(f"Output file: {output_tif_path}")

# Check if file already exists
if output_tif_path.exists():
    print(f"\n✓ Output file already exists: {output_tif_path}")
    print(f"  Skipping downsampling process.")
    
    # Verify existing file
    with rasterio.open(output_tif_path) as verify:
        print(f"\nExisting file properties:")
        print(f"  Dimensions: {verify.width} x {verify.height}")
        print(f"  Resolution: {abs(verify.transform[0]):.8f} degrees")
        print(f"  Bounds: {verify.bounds}")
else:
    print(f"\nOutput file does not exist. Starting downsampling...")
    
    downsample_factor = 4

    def downsample_categorical_raster(data, factor, nodata=None):
        """
        Downsample categorical raster using mode (most common value).
        Fully vectorized implementation using numpy.apply_along_axis.
        
        Args:
            data: 2D numpy array to downsample
            factor: Downsampling factor (e.g., 4 means 4x4 pixels -> 1 pixel)
            nodata: NoData value to exclude from mode calculation
        
        Returns:
            Downsampled 2D numpy array
        """
        height, width = data.shape
        new_height = height // factor
        new_width = width // factor
        
        # Crop to exact multiple of factor
        crop_height = new_height * factor
        crop_width = new_width * factor
        data_cropped = data[:crop_height, :crop_width]
        
        # Reshape to group pixels into blocks: (new_h, factor, new_w, factor)
        reshaped = data_cropped.reshape(new_height, factor, new_width, factor)
        
        # Transpose to: (new_h, new_w, factor, factor)
        blocks = reshaped.transpose(0, 2, 1, 3)
        
        # Flatten each block: (new_h, new_w, factor*factor)
        blocks_flat = blocks.reshape(new_height, new_width, factor * factor)
        
        # Use sentinel-based approach for maximum speed
        # Replace nodata with sentinel, use vectorized mode, then restore nodata
        if nodata is not None:
            # Choose sentinel value that doesn't conflict with data
            if np.issubdtype(data.dtype, np.integer):
                # For integer types, use max possible value + 1
                dtype_info = np.iinfo(data.dtype)
                sentinel = dtype_info.max
            else:
                # For float, use a large negative number
                sentinel = -999999.0
            
            # Create mask of all-nodata blocks
            all_nodata_mask = np.all(blocks_flat == nodata, axis=2)
            
            # Replace nodata with sentinel temporarily
            blocks_processed = np.where(blocks_flat == nodata, sentinel, blocks_flat)
            
            # Use vectorized scipy.stats.mode (much faster!)
            print(f"    Calculating mode (vectorized) for {new_height * new_width:,} pixels...")
            mode_result = stats.mode(blocks_processed, axis=2, keepdims=False, nan_policy='omit')
            result = mode_result.mode.astype(data.dtype)
            
            # Restore nodata where all values were nodata
            result[all_nodata_mask] = nodata
            
            # Also check if mode is sentinel (means all values in block were nodata)
            # This handles edge cases where sentinel might accidentally be the mode
            if np.issubdtype(data.dtype, np.integer):
                sentinel_check = result == sentinel
            else:
                sentinel_check = np.isclose(result, sentinel)
            result[sentinel_check] = nodata
        else:
            # No nodata - fully vectorized
            print(f"    Calculating mode (vectorized) for {new_height * new_width:,} pixels...")
            mode_result = stats.mode(blocks_flat, axis=2, keepdims=False)
            result = mode_result.mode.astype(data.dtype)
        
        return result

    with rasterio.open(tif_path) as src:
        print(f"\nSource TIF properties:")
        print(f"  Dimensions: {src.width} x {src.height}")
        print(f"  Resolution: {abs(src.transform[0]):.8f} degrees")
        
        # Calculate new dimensions
        new_width = src.width // downsample_factor
        new_height = src.height // downsample_factor
        new_res = abs(src.transform[0]) * downsample_factor
        
        print(f"\nTarget TIF properties:")
        print(f"  Dimensions: {new_width} x {new_height}")
        print(f"  Resolution: {new_res:.8f} degrees")
        print(f"  Downsample factor: {downsample_factor}x")
        
        # Calculate new transform (resolution increased by factor)
        new_transform = rasterio.Affine(
            src.transform[0] * downsample_factor,  # pixel width
            src.transform[1],
            src.transform[2],
            src.transform[3],
            src.transform[4] * downsample_factor,  # pixel height (negative)
            src.transform[5]
        )
        
        # Create output profile
        profile = src.profile.copy()
        profile.update({
            'width': new_width,
            'height': new_height,
            'transform': new_transform,
            'compress': 'lzw',
            'tiled': True
        })
        
        print(f"\nDownsampling using mode (most common value) for categorical data...")
        print(f"Reading full raster into memory...")
        
        # Initialize timing
        start_time = time.time()
        
        with rasterio.open(output_tif_path, 'w', **profile) as dst:
            for band_idx in range(1, src.count + 1):
                band_start = time.time()
                print(f"\nProcessing band {band_idx}/{src.count}...")
                
                # Read entire band at once (faster than blocks for this operation)
                print(f"  Reading source data...")
                data = src.read(band_idx)
                read_time = time.time() - band_start
                print(f"  Read complete ({read_time:.1f}s)")
                
                # Downsample using vectorized mode calculation
                print(f"  Downsampling (this may take a few minutes)...")
                downsample_start = time.time()
                nodata_val = src.nodata
                
                downsampled_data = downsample_categorical_raster(
                    data, 
                    downsample_factor, 
                    nodata_val
                )
                
                downsample_time = time.time() - downsample_start
                print(f"  Downsampling complete ({downsample_time:.1f}s)")
                
                # Write downsampled data
                print(f"  Writing to file...")
                dst.write(downsampled_data, band_idx)
                write_time = time.time() - downsample_start - downsample_time
                
                elapsed_time = time.time() - band_start
                print(f"  ✓ Band {band_idx} complete (total: {elapsed_time:.1f}s)")
                print(f"    Breakdown: read={read_time:.1f}s, downsample={downsample_time:.1f}s, write={write_time:.1f}s")
        
        print(f"\n✓ Created downsampled TIF file: {output_tif_path}")
        
        # Verify output
        with rasterio.open(output_tif_path) as verify:
            print(f"\nVerification:")
            print(f"  Dimensions: {verify.width} x {verify.height}")
            print(f"  Resolution: {abs(verify.transform[0]):.8f} degrees")
            print(f"  Bounds: {verify.bounds}")


DOWNSAMPLING TIF FILE

Output directory: ../tests/tests_data/hazards/Fire/land_cover
Output file: ../tests/tests_data/hazards/Fire/land_cover/2024_brazil_land_cover.tif

Output file does not exist. Starting downsampling...

Source TIF properties:
  Dimensions: 167551 x 144835
  Resolution: 0.00026949 degrees

Target TIF properties:
  Dimensions: 41887 x 36208
  Resolution: 0.00107798 degrees
  Downsample factor: 4x

Downsampling using mode (most common value) for categorical data...
Reading full raster into memory...

Processing band 1/1...
  Reading source data...
  Read complete (104.3s)
  Downsampling (this may take a few minutes)...
